In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/My Drive

/content/gdrive/My Drive


In [ ]:
!pip install simplejson

In [ ]:
# Dataframe
import pandas as pd

# Array
import numpy as np

# Decompress the file
import gzip

# Visualizations
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import matplotlib.colors as colors
%matplotlib inline

# Datetime
from datetime import datetime

## Warnings
import warnings
from scipy import stats
warnings.filterwarnings('ignore')

In [ ]:
!pip install simplejson

In [ ]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

**Importing GoodreadsBook json file**

In [ ]:
import pandas as pd
import gzip
import json
def parse(path):
  g = open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
    if(i>=400000):
      break
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('goodreads_books.json')

In [ ]:
df

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,0312853122,1,[],US,,"[{'count': '3', 'name': 'to-read'}, {'count': ...",,false,4.00,,[],,Paperback,https://www.goodreads.com/book/show/5333265-w-...,"[{'author_id': '604031', 'role': ''}]",St. Martin's Press,256,1,9780312853129,9,,1984,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3,5400751,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film
1,0743509986,6,[],US,,"[{'count': '2634', 'name': 'to-read'}, {'count...",,false,3.23,B000FC0PBC,"[8709549, 17074050, 28937, 158816, 228563, 112...","Anita Diamant's international bestseller ""The ...",Audio CD,https://www.goodreads.com/book/show/1333909.Go...,"[{'author_id': '626222', 'role': ''}]",Simon & Schuster Audio,,1,9780743509985,10,Abridged,2001,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,10,1323437,Good Harbor,Good Harbor
2,,7,[189911],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,false,4.03,,"[19997, 828466, 1569323, 425389, 1176674, 2627...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600,,,,Book Club Edition,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,8948723,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,0743294297,3282,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",,false,3.49,B002ENBLOK,"[6604176, 6054190, 2285777, 82641, 7569453, 70...",Addie Downs and Valerie Adler were eight when ...,Hardcover,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",Atria Books,368,14,9780743294294,7,,2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,6243154,Best Friends Forever,Best Friends Forever
4,0850308712,5,[],US,,"[{'count': '32', 'name': 'to-read'}, {'count':...",,false,3.40,,[],,,https://www.goodreads.com/book/show/287140.Run...,"[{'author_id': '149918', 'role': ''}]",,,,9780850308716,,,,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15,278577,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,3939080314,1,[],US,,[],,false,3.50,,[],,,https://www.goodreads.com/book/show/8047305-di...,"[{'author_id': '3486727', 'role': ''}]",,,,9783939080312,,,,https://www.goodreads.com/book/show/8047305-di...,https://s.gr-assets.com/assets/nophoto/book/11...,8047305,2,12691764,Die Sechs Schüsse Von Philadelphia,Die Sechs Schüsse Von Philadelphia
399996,0002446677,2,[220559],US,eng,"[{'count': '7', 'name': 'to-read'}, {'count': ...",,false,3.76,,[],"The girl in the dock, Camilla Barnard, was alm...",Hardcover,https://www.goodreads.com/book/show/1735650.Pa...,"[{'author_id': '59132', 'role': ''}]",Harper,230,,9780002446679,,,1970,https://www.goodreads.com/book/show/1735650.Pa...,https://images.gr-assets.com/books/1395765682m...,1735650,16,1733162,"Past Praying For (Antony Maitland, #14)","Past Praying For (Antony Maitland, #14)"
399997,0470154594,7,[],US,,"[{'count': '42', 'name': 'to-read'}, {'count':...",,false,3.10,B00UKFX5EI,[],How many of us never meet the person with whom...,Paperback,https://www.goodreads.com/book/show/2609029-wh...,"[{'author_id': '569879', 'role': ''}]",John Wiley & Sons,271,1,9780470154595,1

In [ ]:
df.columns

Index(['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin',
       'similar_books', 'description', 'format', 'link', 'authors',
       'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information', 'publication_year', 'url',
       'image_url', 'book_id', 'ratings_count', 'work_id', 'title',
       'title_without_series'],
      dtype='object')

**Dropping unwanted columns which are not needed for our hybrid system**

In [ ]:
# Most of these columns had null values so we are dropping them
df=df.drop(['isbn', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'kindle_asin',
       'similar_books', 'format','publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information','work_id'],axis=1)

In [ ]:
df.head()

,text_reviews_count,average_rating,description,link,authors,publication_year,url,image_url,book_id,ratings_count,title,title_without_series
0,1,4.00,,https://www.goodreads.com/book/show/5333265-w-...,"[{'author_id': '604031', 'role': ''}]",1984,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film
1,6,3.23,"Anita Diamant's international bestseller ""The ...",https://www.goodreads.com/book/show/1333909.Go...,"[{'author_id': '626222', 'role': ''}]",2001,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,10,Good Harbor,Good Harbor
2,7,4.03,Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]",1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,3282,3.49,Addie Downs and Valerie Adler were eight when ...,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,Best Friends Forever,Best Friends Forever
4,5,3.40,,https://www.goodreads.com/book/show/287140.Run...,"[{'author_id': '149918', 'role': ''}]",,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...


In [ ]:
df.columns

Index(['text_reviews_count', 'average_rating', 'description', 'link',
       'authors', 'publication_year', 'url', 'image_url', 'book_id',
       'ratings_count', 'title', 'title_without_series'],
      dtype='object')

**Dropping rows with Repeated Titles**

*Some of the books had same title with different book id so we are dropping those titles.*

In [ ]:
df= df.drop_duplicates(subset = ["title"])

In [ ]:
df.shape

(359916, 12)

**Extracting books with only English Language**

*Some of the books where in multiple languages, for this project we are taking the books only in english language*

In [ ]:
!pip install langdetect

     |████████████████████████████████| 983kB 6.9MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993195 sha256=e3e3b11a7eb5d8fb57e8d7a0dfa9768e2d38a9c3a3f3538fecc8ed3b53641a81
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [ ]:
from langdetect import detect

In [ ]:
def language(text):
  from langdetect import detect
  try:
    lang=detect(str(text))
    return lang
  except:
    return 'NULL'

In [ ]:
df['language'] = df['title'].apply(language)

In [ ]:
df.language.value_counts()

en       237753
de        18002
es         9666
it         9555
ar         8361
fr         7700
nl         6101
id         6015
pt         5167
af         3924
tl         3496
no         3211
ca         3087
da         2986
ro         2887
fi         2788
tr         2722
sv         2509
fa         2419
et         2318
el         1710
pl         1596
cy         1443
so         1270
sw         1226
cs         1161
ru         1068
bg         1056
hr         1015
lt          944
sl          876
vi          832
hu          632
sk          619
bn          577
th          504
lv          434
ja          398
sq          272
ur          233
mk          222
ko          218
he          200
NULL        172
uk          165
ta           94
ml           92
zh-cn        84
te           40
zh-tw        34
hi           20
mr           19
ne           10
kn           10
gu            3
Name: language, dtype: int64

In [ ]:
df=df[df['language']=='en']

In [ ]:
df

,text_reviews_count,average_rating,description,link,authors,publication_year,url,image_url,book_id,ratings_count,title,title_without_series,language
0,1,4.00,,https://www.goodreads.com/book/show/5333265-w-...,"[{'author_id': '604031', 'role': ''}]",1984,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film,en
2,7,4.03,Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]",1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ...",en
4,5,3.40,,https://www.goodreads.com/book/show/287140.Run...,"[{'author_id': '149918', 'role': ''}]",,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...,en
5,7,4.13,"Relates in vigorous prose the tale of Aeneas, ...",https://www.goodreads.com/book/show/287141.The...,"[{'author_id': '3041852', 'role': ''}]",2006,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46,The Aeneid for Boys and Girls,The Aeneid for Boys and Girls,en
6,4,4.38,,https://www.goodreads.com/book/show/378460.The...,"[{'author_id': '215594', 'role': ''}]",1979,https://www.goodreads.com/book/show/378460.The...,https://s.gr-assets.com/assets/nophoto/book/11...,378460,12,The Wanting of Levine,The Wanting of Levine,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399994,7,3.58,Werewolf Sabrina Kelly longs to put her family...,https://www.goodreads.com/book/show/9776119-co...,"[{'author_id': '259021', 'role': ''}]",,https://www.goodreads.com/book/show/9776119-co...,https://s.gr-assets.com/assets/nophoto/book/11...,9776119,69,"Courage of the Wolf (Draicon Werewolves, #8)","Courage of the Wolf (Draicon Werewolves, #8)",en
399996,2,3.76,"The girl in the dock, Camilla Barnard, was alm...",https://www.goodreads.com/book/show/1735650.Pa...,"[{'author_id': '59132', 'role': ''}]",1970,https://www.goodreads.com/book/show/1735650.Pa...,https://images.gr-assets.com/books/1395765682m...,1735650,16,"Past Praying For (Antony Maitland, #14)","Past Praying For (Antony Maitland, #14)",en
399997,7,3.10,How many of us never meet the person with whom...,https://www.goodreads.com/book/show/2609029-wh...,"[{'author_id': '569879', 'role': ''}]",2008,https://www.goodreads.com/book/show/2609029-wh...,https://s.gr-assets.com/assets/nophoto/book/11...,2609029,51,Why Mr. Right Can't Find You: The Surprising A...,Why Mr. Right Can't Find You: The Surprising A...,en
399998,1,5.00,This work has been selected by scholars as bei...,https://www.goodreads.com/book/show/32087036-a...,"[{'author_id': '15794963', 'role': ''}]",2016,https://www.goodreads.com/book/show/32087036-a...,https://images.gr-assets.com/books/1474570280m...,32087036,1,Adoniram Judson: A Biography,Adoniram Judson: A Biography,en


In [ ]:
#description should not have a min lenght for nlp topic modelling
df=df[(df.description.str.len() > 10)]

In [ ]:
df.shape

(193680, 13)

In [ ]:
df['language'] = df['description'].apply(language)

In [ ]:
df

,text_reviews_count,average_rating,description,link,authors,publication_year,url,image_url,book_id,ratings_count,title,title_without_series,language
2,7,4.03,Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]",1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ...",en
5,7,4.13,"Relates in vigorous prose the tale of Aeneas, ...",https://www.goodreads.com/book/show/287141.The...,"[{'author_id': '3041852', 'role': ''}]",2006,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46,The Aeneid for Boys and Girls,The Aeneid for Boys and Girls,en
7,6,4.22,"To Kara's astonishment, she discovers that a p...",https://www.goodreads.com/book/show/6066812-al...,"[{'author_id': '19158', 'role': ''}]",2009,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,6066812,98,All's Fairy in Love and War (Avalon: Web of Ma...,All's Fairy in Love and War (Avalon: Web of Ma...,en
8,4,3.86,Secrets. Sometimes keeping them in confidence ...,https://www.goodreads.com/book/show/34883016-p...,"[{'author_id': '5807700', 'role': ''}]",2017,https://www.goodreads.com/book/show/34883016-p...,https://images.gr-assets.com/books/1493525974m...,34883016,5,Playmaker: A Venom Series Novella,Playmaker: A Venom Series Novella,en
9,39,3.81,"Wisdom, humor, and dark observations by the fo...",https://www.goodreads.com/book/show/287149.The...,"[{'author_id': '2983296', 'role': ''}, {'autho...",2000,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,287149,986,The Devil's Notebook,The Devil's Notebook,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399994,7,3.58,Werewolf Sabrina Kelly longs to put her family...,https://www.goodreads.com/book/show/9776119-co...,"[{'author_id': '259021', 'role': ''}]",,https://www.goodreads.com/book/show/9776119-co...,https://s.gr-assets.com/assets/nophoto/book/11...,9776119,69,"Courage of the Wolf (Draicon Werewolves, #8)","Courage of the Wolf (Draicon Werewolves, #8)",en
399996,2,3.76,"The girl in the dock, Camilla Barnard, was alm...",https://www.goodreads.com/book/show/1735650.Pa...,"[{'author_id': '59132', 'role': ''}]",1970,https://www.goodreads.com/book/show/1735650.Pa...,https://images.gr-assets.com/books/1395765682m...,1735650,16,"Past Praying For (Antony Maitland, #14)","Past Praying For (Antony Maitland, #14)",en
399997,7,3.10,How many of us never meet the person with whom...,https://www.goodreads.com/book/show/2609029-wh...,"[{'author_id': '569879', 'role': ''}]",2008,https://www.goodreads.com/book/show/2609029-wh...,https://s.gr-assets.com/assets/nophoto/book/11...,2609029,51,Why Mr. Right Can't Find You: The Surprising A...,Why Mr. Right Can't Find You: The Surprising A...,en
399998,1,5.00,This work has been selected by scholars as bei...,https://www.goodreads.com/book/show/32087036-a...,"[{'author_id': '15794963', 'role': ''}]",2016,https://www.goodreads.com/book/show/32087036-a...,https://images.gr-assets.com/books/1474570280m...,32087036,1,Adoniram Judson: A Biography,Adoniram Judson: A Biography,en


In [ ]:
df.language.value_counts()

en      189625
id         996
fr         518
it         490
es         462
de         418
pt         231
nl         159
tr          70
tl          65
hr          64
sv          63
fi          55
pl          50
sk          49
cy          49
et          35
da          35
no          35
sl          35
ro          30
cs          30
af          24
NULL        19
so          18
hu          17
ca          15
vi          10
lt           7
sw           5
sq           1
Name: language, dtype: int64

In [ ]:
df=df[df['language']=='en']

In [ ]:
df.shape

(189625, 13)

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
df

,text_reviews_count,average_rating,description,link,authors,publication_year,url,image_url,book_id,ratings_count,title,title_without_series,language
0,7,4.03,Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]",1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ...",en
1,7,4.13,"Relates in vigorous prose the tale of Aeneas, ...",https://www.goodreads.com/book/show/287141.The...,"[{'author_id': '3041852', 'role': ''}]",2006,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46,The Aeneid for Boys and Girls,The Aeneid for Boys and Girls,en
2,6,4.22,"To Kara's astonishment, she discovers that a p...",https://www.goodreads.com/book/show/6066812-al...,"[{'author_id': '19158', 'role': ''}]",2009,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,6066812,98,All's Fairy in Love and War (Avalon: Web of Ma...,All's Fairy in Love and War (Avalon: Web of Ma...,en
3,4,3.86,Secrets. Sometimes keeping them in confidence ...,https://www.goodreads.com/book/show/34883016-p...,"[{'author_id': '5807700', 'role': ''}]",2017,https://www.goodreads.com/book/show/34883016-p...,https://images.gr-assets.com/books/1493525974m...,34883016,5,Playmaker: A Venom Series Novella,Playmaker: A Venom Series Novella,en
4,39,3.81,"Wisdom, humor, and dark observations by the fo...",https://www.goodreads.com/book/show/287149.The...,"[{'author_id': '2983296', 'role': ''}, {'autho...",2000,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,287149,986,The Devil's Notebook,The Devil's Notebook,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
189620,7,3.58,Werewolf Sabrina Kelly longs to put her family...,https://www.goodreads.com/book/show/9776119-co...,"[{'author_id': '259021', 'role': ''}]",,https://www.goodreads.com/book/show/9776119-co...,https://s.gr-assets.com/assets/nophoto/book/11...,9776119,69,"Courage of the Wolf (Draicon Werewolves, #8)","Courage of the Wolf (Draicon Werewolves, #8)",en
189621,2,3.76,"The girl in the dock, Camilla Barnard, was alm...",https://www.goodreads.com/book/show/1735650.Pa...,"[{'author_id': '59132', 'role': ''}]",1970,https://www.goodreads.com/book/show/1735650.Pa...,https://images.gr-assets.com/books/1395765682m...,1735650,16,"Past Praying For (Antony Maitland, #14)","Past Praying For (Antony Maitland, #14)",en
189622,7,3.10,How many of us never meet the person with whom...,https://www.goodreads.com/book/show/2609029-wh...,"[{'author_id': '569879', 'role': ''}]",2008,https://www.goodreads.com/book/show/2609029-wh...,https://s.gr-assets.com/assets/nophoto/book/11...,2609029,51,Why Mr. Right Can't Find You: The Surprising A...,Why Mr. Right Can't Find You: The Surprising A...,en
189623,1,5.00,This work has been selected by scholars as bei...,https://www.goodreads.com/book/show/32087036-a...,"[{'author_id': '15794963', 'role': ''}]",2016,https://www.goodreads.com/book/show/32087036-a...,https://images.gr-assets.com/books/1474570280m...,32087036,1,Adoniram Judson: A Biography,Adoniram Judson: A Biography,en


In [ ]:
final_book=df

In [ ]:
#pip install xlsxwriter

     |████████████████████████████████| 153kB 6.5MB/s 


In [ ]:
#import xlsxwriter

In [ ]:
#writer = pd.ExcelWriter(r'cleaned_goodreadsbook#1.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
#final_book.to_excel(writer)
#writer.close()